# De l’étiquetage avec *TreeTagger*

## Définition

Opération par laquelle un programme associe automatiquement à un mot des étiquettes grammaticales, comme :

- la partie du discours (*N*)
- le genre (*NF*)
- le nombre (*NFP*)
- …

Elle intervient après celle de segmentation en mots et se positionne comme pré-requis pour l’analyse syntaxique de surface. Le résultat est un couple (mot, étiquette) :

- *Le petit chat boit du lait.*
- *Le/DET petit/ADJ chat/N boit/V du/DET lait/N ./PONCT*

## Des mots résolument ambigus

Certaines étiquettes étant en concurrence pour un mot, leur attribution est rarement aisée. Considérons par exemple la phrase ci-dessous :

> La petite brise la glace.

Elle peut s’interpréter de deux façons : soit on imagine une petite fille en colère brisant un miroir, soit on se place dans un contexte hivernal où le vent fait trembler notre héroïne :

```mermaid
---
title: La colère d’une petite fille
---
flowchart TB
  direction TB
  A("La"):::formal-->At1["DET"]:::academic
  B("petite"):::formal-->Bt1["N"]:::academic
  C("brise"):::formal-->Ct1["V"]:::academic
  D("la"):::formal-->Dt1["DET"]:::academic
  E("glace"):::formal-->Et1["N"]:::academic
  F("."):::formal-->Ft1["PONCT"]:::academic
classDef formal fill:#fff,stroke:#D68738,color:#D68738
classDef academic fill:#032B4F,stroke:#032B4F,color:#fff
```

```mermaid
---
title: Les déboires d’une aventurière
---
flowchart TB
  direction TB
  A2("La"):::formal-->At2["DET"]:::academic
  B2("petite"):::formal-->Bt2["ADJ"]:::academic
  C2("brise"):::formal-->Ct2["N"]:::academic
  D2("la"):::formal-->Dt2["PRO"]:::academic
  E2("glace"):::formal-->Et2["V"]:::academic
  F2("."):::formal-->Ft2["PONCT"]:::academic
classDef formal fill:#fff,stroke:#D68738,color:#D68738
classDef academic fill:#032B4F,stroke:#032B4F,color:#fff
```

Cet énoncé est caractéristique des difficultés qu’un étiqueteur peut rencontrer, la très grande majorité des mots et des phrases du français étant ambiguë. Pourquoi alors ne le percevons-nous pas ainsi ? Pourquoi avons-nous l’impression au quotidien que la très grande majorité des énoncés est limpide ? Notre cerveau est en fait très doué à résoudre l’ambiguïté en fonction du contexte avant même qu’elle se présente à nous.

Imaginons deux contextes d’élocution où dans le premier l’orateur fait une pause après « La petite » et l’autre où la pause est positionnée directement après « La » alors que « petite brise » est prononcée dans un seul élan. Dans ces cas, nous attribuons intuitivement une fonction aux mots afin d’établir le sens de la phrase. Mieux, si nous disposons en plus d’un contexte plus large, nous relevons à peine l’ambiguïté puisque la phrase s’inscrit dans une histoire.

Pour un étiqueteur, l’analyse de la phrase revient à prendre une décision entre deux voies possibles : l’une ouverte par l’interprétation de *petite* comme un nom et l’autre comme un adjectif.

```mermaid
flowchart LR
    A("La
    DET"):::academic-->B("petite
    N"):::formal
    B-->C("brise
    V"):::formal
    C-->D("la
    DET"):::formal
    D-->E("glace
    N"):::formal
    E-->F(".
    PONCT"):::academic
    A-->G("petite
    ADJ"):::academic
    G-->H("brise
    N"):::academic
    H-->I("la
    PRO"):::academic
    I-->J("glace
    V"):::academic
    J-->F
    classDef formal fill:#fff,stroke:#D68738,color:#D68738
    classDef academic fill:#032B4F,stroke:#032B4F,color:#fff
```

## *TreeTagger*, un étiqueteur pré-entraîné

*TreeTagger* est un étiqueteur morpho-syntaxique pré-entraîné pour de multiples langues (anglais, allemand, français, espagnol, italien…). Il est développé à [l’IMS de Stuttgart](https://www.ims.uni-stuttgart.de/) par Helmut Schmid. Sans être le meilleur, il est toutefois toujours plébiscité pour des raisons historiques, pour sa rapidité et pour sa couverture linguistique. Parmi les reproches qui persistent pour sa partie française, nous retenons l’absence d’information sur le corpus d’apprentissage ou sur le lexique utilisé.

La procédure d’installation de l’utilitaire et de ses fichiers de paramètres est détaillée dans [la documentation en ligne](https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/).

### Annotation des tokens

#### Étiquetage en parties du discours

La procédure d’étiquetage de *TreeTagger* implémente un modèle probabiliste qui, pour des raisons de performance, prend la forme d’un arbre de décision. Différents modèles de langage sont disponibles sur la page des téléchargements. Il s’agit de fichiers de paramètres qui portent une extension en *.par* :

In [ ]:
!echo "A Lannister always pays his debts." \
| perl ./TT/utf8-tokenize.perl -e \
| ./TT/tree-tagger ./TT/english.par

L’ajout d’une option `-token` permet de révéler le token étiqueté :

In [ ]:
!echo "A Lannister always pays his debts." \
| perl ./TT/utf8-tokenize.perl -e \
| ./TT/tree-tagger -token ./TT/english.par

Les étiquettes attribuées dépendent du modèle de langage indiqué :

In [ ]:
!echo "A Lannister always pays his debts." \
| perl ./TT/utf8-tokenize.perl -e \
| ./TT/tree-tagger -token ./TT/french.par

*TreeTagger* ne vérifie donc pas la vraisemblance des tokens en entrée avec le modèle de langage spécifié. La pertinence de certains résultats peut s’en ressentir !

**Remarque :** pour prendre connaissance de la signification des étiquettes, se référer à la documentation en ligne.

#### Lemmatisation

Par lemmatisation, on entend l’opération consistant à obtenir la forme canonique d’un mot. Par exemple, *parent* serait le lemme de *parents*, ou *chanter* celui de la forme fléchie *chantes*.

L’option `-lemma` active la fonction :

In [ ]:
!echo "A Lannister always pays his debts." \
| perl ./TT/utf8-tokenize.perl -e \
| ./TT/tree-tagger -token -lemma ./TT/english.par

Pour *TreeTagger*, cette étape n’est résolue que par extraction depuis un lexique où à chaque lemme est associé une étiquette. En conséquence, si dans la phrase suivante il parvient bien à désambiguiser les occurrences du mot *dérive*, c’est uniquement grâce à leurs étiquettes :

In [ ]:
!echo "Le navire dérive à cause d'une dérive instable." \
| perl ./TT/utf8-tokenize.perl -f \
| ./TT/tree-tagger -token -lemma ./TT/french.par

*TreeTagger* ne fait aucune hypothèse quant aux lemmes aussi, en cas de doute, il liste les différentes possibilités avec une barre verticale :

In [ ]:
!echo "Il a été secouru par le fils du jardinier." \
| perl ./TT/utf8-tokenize.perl -f \
| ./TT/tree-tagger -token -lemma ./TT/french.par

Étiqueté comme *NOM*, le mot *fils* peut aussi bien être le pluriel de *fil* – comme dans *le fil de l’histoire* ou *un fil de lin* – que l’une des formes du lemme *fils*.

### Frontières de décision

Considérant les ambiguïtés inhérentes aux langues et le modèle probabiliste derrière *TreeTagger*, rien de surprenant à ce qu’il effectue un choix au regard du contexte.

Prenons un énoncé qui ne pose guère de problème :

In [ ]:
!echo "Le capitaine Grant a mené le bataillon au combat." \
| perl ./TT/utf8-tokenize.perl -f \
| ./TT/tree-tagger -token -lemma ./TT/french.par

Ici, bien qu’inconnu, *Grant* est bien considéré comme un nom propre (*NAM*) grâce à la présence de la majuscule. Retirons à *TreeTagger* cette faculté de s’appuyer sur le préfixe avec l’option `-ignore-prefix` :

In [ ]:
!echo "Le capitaine Grant a mené le bataillon au combat." \
| perl ./TT/utf8-tokenize.perl -f \
| ./TT/tree-tagger -token -ignore-prefix ./TT/french.par

*Grant* est maintenant considéré comme un adjectif, un choix logique quand on le sait précédé par un nom.

Pour comprendre le mécanisme, il nous faut révéler les frontières de décision avec les options `-threshold` et `-prob`. Le seuil est défini dans l’intervalle $]0,1]$ :

In [ ]:
!echo "Le capitaine Grant a mené le bataillon au combat." \
| perl ./TT/utf8-tokenize.perl -f \
| ./TT/tree-tagger -token -ignore-prefix -threshold 0.01 -prob ./TT/french.par

Pour la majorité des mots de la phrase, *TreeTagger* n’a aucun doute. Il est en revanche moins assuré pour le fameux *Grant* qu’il identifie d’ailleurs à 71 % comme un participe présent (*VER:ppre*) à cause de sa terminaison en *-ant*.

Pourquoi, alors, choisit-il au final de l’étiqueter *ADJ* ? Les probabilités affichées sont en fait celles du token considéré sans le contexte et, malheureusement, *TreeTagger* n’expose pas les probabilités conditionnelles calculées avec le contexte.

Promouvons notre capitaine au rang de général :

In [ ]:
!echo "Le général Grant a mené le bataillon au combat." \
| perl ./TT/utf8-tokenize.perl -f \
| ./TT/tree-tagger -token -threshold 0.01 -prob ./TT/french.par

Le mot *général* devient pour lui un adjectif, ce qui met en balance l’étiquette de *Grant*. Au final, même en lui autorisant l’appui du préfixe, *TreeTagger* fait également le mauvais choix pour *Grant* :

In [ ]:
!echo "Le général Grant a mené le bataillon au combat." \
| perl ./TT/utf8-tokenize.perl -f \
| ./TT/tree-tagger -token ./TT/french.par

### Définir un modèle de langage

Comme les modèles de langage sont décrits dans des fichiers binaires en *.par*, nous n’intervenons pas directement dessus pour leur apporter des modifications mais sur les ressources qui ont permis de les générer. *TreeTagger* est en effet fourni avec un utilitaire `train-tree-tager` pour créer un de ces fichiers *.par* à partir de trois ressources :

- un lexique des formes fléchies ;
- un corpus d’apprentissage ;
- une liste d’étiquettes.

#### Le lexique des formes fléchies

Il s’agit d’un fichier plat où chaque ligne contient un mot-forme suivi d’une tabulation puis d’une paire étiquette-lemme eux-mêmes séparés par une espace. En cas d’ambiguïté sur l’étiquetage, il peut exister plusieurs paires qui seront listées derrière des tabulations, comme dans l’exemple :

```csv
fil    N fil
fils   N fils    N fil 
```

#### Le corpus d’apprentissage

À l’aide de ce corpus, *TreeTagger* va apprendre un langage, c’est-à-dire une façon de représenter la langue. Pour cela, il va se reposer sur un unique fichier tabulé constitué des mots-formes et de leur étiquette associée, sans oublier la ponctuation. À l’évidence, l’étiquetage doit être irréprochable.

```csv
Le     DET
petit  ADJ
chat   NOM
boit   VER
du     DET
lait   NOM
.      SENT
```

#### La liste des étiquettes

Dans ce fichier, il suffit simplement de lister les étiquettes séparées par une espace. Lorsque *TreeTagger* sera face à un mot pour lequel il ne trouve aucun lemme, il cherchera parmi cette liste l’étiquette à lui affecter.

```csv
ADJ DET NOM VER
```

#### Entraîner *TreeTagger*

La commande suivante permettra de générer un fichier *basic-model.par* à partir des trois ressources décrites plus haut :

```sh
./TT/train-tree-tagger -utf8 lexicon.txt tags.txt corpus.txt basic-model.par
```